In [1]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime  # Untuk konversi timestamp ke human-readable
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Konfigurasi database tujuan
dest_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
dest_app = firebase_admin.initialize_app(dest_cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [3]:
# Referensi ke data sumber
source_ref = db.reference('/auto_weather_stat/id-03/data')

# Mengambil data dari database sumber
source_data = source_ref.get()

# Mengubah data menjadi DataFrame pandas
if source_data:
    # Mengubah data menjadi DataFrame
    df = pd.DataFrame.from_dict(source_data, orient='index')
    
    # Jika 'timestamp' sudah ada, jangan pindahkan indeks ke kolom
    if 'timestamp' not in df.columns:
        df.index.name = 'timestamp'  # Mengatur nama indeks
        df.reset_index(inplace=True)  # Memindahkan indeks menjadi kolom biasa
    
    print(df.tail())  # Menampilkan data untuk verifikasi
else:
    print("Tidak ada data di jalur /auto_weather_stat/id-03/data.")

In [10]:
df.tail(10)

In [5]:
df.info()

In [6]:
df = df[(df['temperature'] >= 10) & 
                      (df['humidity'] >= 0) & 
                      (df['pressure'] >= 1000) & 
                      (df['dew'] >= 10) &
                      (df['volt'] >= 0)]

In [20]:
# Membuat DataFrame awal
data = pd.DataFrame(df)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Membuat rentang waktu yang lengkap dengan interval per menit
complete_range = pd.date_range(start=data.index.min(), end=df.index.max(), freq='T')

# Reindexing dengan rentang waktu lengkap, variabel lain akan terisi NaN
df = df.reindex(complete_range)

# Menampilkan hasil
print(df.head(15))

In [8]:
# Plot line chart
df.plot(y=['dew', 'humidity', 'pressure', 'temperature', 'volt'], subplots=True, figsize=(10, 15))
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>